
#### Exercise 8.1

In [67]:
import nltk
import numpy as np
import hmmlearn, hmmlearn.hmm

In [24]:
temp_file = open('hmm_sentences.txt','r', encoding='utf-8', errors='ignore')
text_lines = temp_file.readlines()

In [28]:
len(text_lines)

1000

In [29]:
hmm_sentences = []
for line in text_lines:
    temp_text = line.strip()
    temp_text = ' '.join(temp_text.split())
    hmm_sentences.append(temp_text)

In [31]:
hmm_sentences[0]

'where may you feed ?'

In [32]:
# Tokenize downloaded texts and change them to NLTK format
mydownloaded_nltk_texts = []
for k in range(len(hmm_sentences)):
    temp_tokenizedtext = nltk.word_tokenize(hmm_sentences[k])
    temp_nltktext = nltk.Text(temp_tokenizedtext)
    mydownloaded_nltk_texts.append(temp_nltktext)

In [34]:
len(mydownloaded_nltk_texts)

1000

In [36]:
# Make all downloaded texts lowercase
mydownloaded_lowercase_texts = []
for k in range(len(mydownloaded_nltk_texts)):
    temp_lowercase_text = []
    for l in range(len(mydownloaded_nltk_texts[k])):
        lowercase_word = mydownloaded_nltk_texts[k][l].lower()
        temp_lowercase_text.append(lowercase_word)
    temp_lowercasetest = nltk.Text(temp_lowercase_text)
    mydownloaded_lowercase_texts.append(temp_lowercase_text)

In [38]:
len(mydownloaded_lowercase_texts)

1000

In [39]:
# Convert a POS tag for WordNet
def tagtowordnet(postag):
    wordnettag=-1
    if postag[0]=='N':
        wordnettag='n'
    elif postag[0]=='V':
        wordnettag='v'
    elif postag[0]=='J':
        wordnettag='a'
    elif postag[0]=='R':
        wordnettag='r'
    return(wordnettag)

In [43]:
# POS tag and lemmatize the loaded texts
# Download tagger and wordnet resources if you do not have them already
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# lemmatize downloaded texts
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatizetext(nltktexttolemmatize):
    # tag the text with POS tags
    taggedtext = nltk.pos_tag(nltktexttolemmatize)
    # lemmatize each word text
    lemmatizedtext = []
    for l in range(len(taggedtext)):
        # Lemmatize a word using the WordNet converted POS tag
        wordtolemmatize = taggedtext[l][0]
        wordnettag = tagtowordnet(taggedtext[l][1])
        if wordnettag != -1:
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize, wordnettag)
        else:
            lemmatizedword = wordtolemmatize
        # store the lemmatized word
        lemmatizedtext.append(lemmatizedword)
    return(lemmatizedtext)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tunde\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
mydownloaded_lemmatizedtexts = []
for k in range(len(mydownloaded_lowercase_texts)):
    lemmatizedtext = lemmatizetext(mydownloaded_lowercase_texts[k])
    lemmatizedtext = nltk.Text(lemmatizedtext)
    mydownloaded_lemmatizedtexts.append(lemmatizedtext)

In [48]:
mydownloaded_lemmatizedtexts[0]

<Text: where may you fee ?...>

In [50]:
# find the vocabularies of each document
myvocabularies = []
myindices_in_vocabularies = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # get unique words and where they occur
    temptext = mydownloaded_lemmatizedtexts[k]
    unique_results = np.unique(temptext, return_inverse = True)
    unique_words = unique_results[0]
    word_indices = unique_results[1]
    # store the vocabularies and the indices
    myvocabularies.append(unique_words)
    myindices_in_vocabularies.append(word_indices)

In [54]:
len(myvocabularies), len(myindices_in_vocabularies)

(1000, 1000)

In [55]:
# unify the vocabularies
# first concatenate all vocabularies
tempvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    tempvocabulary.extend(myvocabularies[k])

# find unique words among all the vocabularies
uniqueresults = np.unique(tempvocabulary, return_inverse = True)
unifiedvocabulary = uniqueresults[0]
wordindices = uniqueresults[1]

In [57]:
len(unifiedvocabulary)

43

In [63]:
# translate previous indices to the unified vocabulary
# must keep track where each vocabulary started in the concatenated one
vocabularystart = 0
myindices_in_unifiedvocabulary = []
for k in range(len(mydownloaded_lemmatizedtexts)):
    # in order to shift word indices, we must temporarily
    # convert their data type to a numpy array
    tempindices = np.array(myindices_in_vocabularies[k])
    tempindices = tempindices + vocabularystart
    tempindices = wordindices[tempindices]
    myindices_in_unifiedvocabulary.append(tempindices)
    vocabularystart = vocabularystart + len(myvocabularies[k])

In [66]:
len(myindices_in_unifiedvocabulary)

1000

In [102]:
wordindices[np.array(myindices_in_vocabularies[0])]

array([38, 27, 42, 15,  2], dtype=int64)

In [106]:
wordindices

array([ 2, 15, 27, ..., 37, 40, 41], dtype=int64)

In [108]:
unifiedvocabulary

array([',', '.', '?', 'a', 'and', 'answer', 'ask', 'be', 'bird', 'but',
       'can', 'cat', 'cautious', 'dog', 'explain', 'fee', 'feed', 'fox',
       'friendly', 'he', 'help', 'her', 'him', 'how', 'however', 'i',
       'insightful', 'may', 'me', 'robot', 'she', 'small', 'strong',
       'the', 'them', 'then', 'they', 'when', 'where', 'why', 'will',
       'wise', 'you'], dtype='<U10')

In [99]:
unifiedvocabulary[myindices_in_unifiedvocabulary[0]]

array(['where', 'may', 'you', 'fee', '?'], dtype='<U10')

In [71]:
concatenated_data = []
documentlengths = []
for k in range(len(myindices_in_unifiedvocabulary)):
    concatenated_data.extend(myindices_in_unifiedvocabulary[k])
    documentlengths.append(len(myindices_in_unifiedvocabulary[k]))
    
concatenated_data = np.matrix(concatenated_data).T

In [87]:
myhmm = hmmlearn.hmm.MultinomialHMM(n_components=5, n_iter=100, verbose=True)
myhmm_fitted = myhmm.fit(concatenated_data, lengths = documentlengths)

         1      -66665.2226             +nan
         2      -60311.2848       +6353.9378
         3      -60288.9410         +22.3438
         4      -60257.8481         +31.0929
         5      -60209.6261         +48.2220
         6      -60131.3479         +78.2782
         7      -60002.8016        +128.5464
         8      -59792.3515        +210.4501
         9      -59448.0693        +344.2822
        10      -58884.7309        +563.3384
        11      -57999.7767        +884.9542
        12      -56768.1312       +1231.6456
        13      -55343.4960       +1424.6352
        14      -54047.9964       +1295.4995
        15      -53083.8300        +964.1665
        16      -52401.7691        +682.0609
        17      -51926.4940        +475.2751
        18      -51620.8999        +305.5941
        19      -51435.8438        +185.0561
        20      -51283.8640        +151.9799
        21      -51079.5148        +204.3492
        22      -50774.0453        +305.4695
        23

In [88]:
myhmm_fitted.startprob_

array([2.14000000e-01, 8.56572050e-42, 2.01843053e-53, 7.86000000e-01,
       0.00000000e+00])

In [89]:
myhmm_fitted.startprob_.shape

(5,)

In [90]:
myhmm_fitted.emissionprob_

array([[2.64693672e-045, 5.25453865e-003, 9.91775680e-002,
        1.24877844e-001, 1.54366359e-002, 7.49706281e-003,
        1.20864044e-003, 4.16285490e-025, 5.63279856e-002,
        1.39802856e-086, 6.30558772e-097, 1.89802845e-002,
        3.21642003e-002, 2.33436817e-002, 4.41693451e-003,
        3.59838959e-003, 5.87738062e-003, 6.30901419e-002,
        4.31877230e-002, 1.67948955e-063, 1.55015967e-003,
        9.74321718e-045, 3.75274990e-028, 0.00000000e+000,
        3.09023929e-125, 8.16861601e-059, 8.06868156e-002,
        2.50699022e-105, 9.23081051e-019, 4.83789281e-002,
        1.82707618e-057, 3.25783510e-002, 9.90823463e-002,
        1.93499319e-001, 7.87244682e-048, 2.54541887e-061,
        2.04171891e-060, 0.00000000e+000, 0.00000000e+000,
        0.00000000e+000, 3.45707663e-107, 3.97850688e-002,
        2.89071165e-017],
       [3.13880641e-001, 2.10120914e-002, 1.01610351e-001,
        1.65146348e-002, 1.61519277e-001, 1.03998727e-061,
        1.78079002e-070, 5.203

In [91]:
np.sum(myhmm_fitted.emissionprob_, axis=1)

array([1., 1., 1., 1., 1.])

In [92]:
myhmm_fitted.emissionprob_.shape

(5, 43)

In [93]:
print(myhmm_fitted.transmat_)

[[7.29069521e-001 1.12538935e-001 9.03278504e-018 1.37186879e-001
  2.12046653e-002]
 [3.33272551e-002 5.52345563e-001 1.05265739e-041 4.14327182e-001
  7.20614422e-101]
 [5.08035868e-002 4.13554787e-116 9.17364624e-002 6.89492648e-179
  8.57459951e-001]
 [6.79418411e-002 6.05357484e-054 9.32058159e-001 1.72491393e-178
  1.38518187e-061]
 [1.48840115e-001 8.51159885e-001 9.58551598e-110 5.94279304e-090
  1.38574006e-045]]


In [94]:
np.sum(myhmm_fitted.transmat_, axis=1)

array([1., 1., 1., 1., 1.])

In [95]:
myhmm_fitted.transmat_.shape

(5, 5)

Yes, the states correspond to meaningful properties of the simplified language.
* The emission probabilities of each state sum to 1.
* The transition probabilities of each state also sum to 1.